In [1]:
#importing necessary libraries 
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import yfinance as yf
from urllib.parse import quote
import seaborn as sns #library not from class
%matplotlib inline

In [2]:
#Load .env environment variables
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
# Set a random seed for reproducibility
np.random.seed(40)

In [4]:
# Set Alpaca API key and secret
APCA_API_KEY_ID = os.getenv("APCA_API_KEY")
APCA_API_SECRET_KEY = os.getenv("APCA_SECRET_KEY")
ALPACA_ENDPOINT_KEY = os.getenv("ALPACA_END_POINT")

#ensuring api keys are correct 
#print(os.getenv("APCA_API_KEY_ID"))
#print(os.getenv("APCA_API_SECRET_KEY"))
#print(os.getenv("ALPACA_ENDPOINT_KEY"))

# Create the Alpaca API object
alpaca = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, api_version="v2",base_url= ALPACA_ENDPOINT_KEY)

## Part 1 - Portfolio Optimization 


In [5]:
#initializing variables
initial_investment = 10000
print(f'The initial investment is: ${initial_investment}')

stock_tickers = ['AAPL', 'AMZN', 'MSFT'] 
ticker_string = ', '.join(stock_tickers)
print(f'The selected stocks are: {stock_tickers}')

start_date = '2020-11-08'  # Set your desired start date
print(f'The start date is: {start_date}')

end_date = '2023-11-08'  # Set your desired start date
print(f'The start date is: {start_date}')

num_simulations = 500 
print(f'The number of monte carlo simulations is: {num_simulations}')

The initial investment is: $10000
The selected stocks are: ['AAPL', 'AMZN', 'MSFT']
The start date is: 2020-11-08
The start date is: 2020-11-08
The number of monte carlo simulations is: 500


### Import S&P500 Data From Alpacas and Display Closing Prices From Chosen Stocks

In [28]:
from urllib.parse import quote

# Importing SP500 
spy = ["SPY"]

# Define the list of stock tickers, excluding SP500 initially
tickersList = stock_tickers

# Set timeframe to '1D'
timeframe = '1D'

# Convert start_date and end_date to ISO format with New York timezone
start_date = pd.Timestamp(start_date, tz='America/New_York').isoformat()
end_date = pd.Timestamp(end_date, tz='America/New_York').isoformat()

# Create an empty DataFrame to store the results for tickers
df_combined_tickers = pd.DataFrame()

# Make the request to Alpaca API for each ticker
for ticker in tickersList:
    # Convert the ticker to URL-encoded format
    ticker_encoded = quote(ticker)
    
    # Make the request to Alpaca API to get bars data
    df_ticker = alpaca.get_bars(ticker_encoded, timeframe, limit=None, start=start_date, end=end_date).df
    
    # Select only the 'close' column and assign the new column name with the ticker symbol for clarity purposes
    df_ticker = df_ticker['close'].rename(ticker)
    
    # Combine the results
    df_combined_tickers = pd.concat([df_combined_tickers, df_ticker], axis=1)

# Display the results (closing prices for chosen tickers)
df_combined_tickers

,AAPL,AMZN,MSFT
2020-11-09 05:00:00+00:00,116.32,3143.74,218.39
2020-11-10 05:00:00+00:00,116.00,3035.02,211.01
2020-11-11 05:00:00+00:00,119.49,3137.39,216.55
2020-11-12 05:00:00+00:00,119.21,3110.28,215.44
2020-11-13 05:00:00+00:00,119.26,3128.81,216.51
...,...,...,...
2023-11-02 04:00:00+00:00,177.57,138.07,348.32
2023-11-03 04:00:00+00:00,176.65,138.60,352.80
2023-11-06 05:00:00+00:00,179.23,139.74,356.53
2023-11-07 05:00:00+00:00,181.82,142.71,360.53


In [33]:
# Calculate daily returns for all ticker's closing prices
daily_returns_tickers = df_combined_tickers.pct_change()
daily_returns_tickers.dropna(inplace=True)

# Display daily returns for the tickers
print("\nDaily Returns for Tickers:")
print(daily_returns_tickers)

# Iterate through each stock in the list of tickers
for stock in stock_tickers:
    # Check if the stock exists in the daily returns columns
    if stock in daily_returns_tickers.columns:
        means[stock] = daily_returns_tickers.mean()[stock]  # Calculate mean for the stock and store in the dictionary
        print(f'{stock} mean: {means[stock]}')  # Print the mean of daily returns for the stock


Daily Returns for Tickers:
                               AAPL      AMZN      MSFT
2020-11-10 05:00:00+00:00 -0.002751 -0.034583 -0.033793
2020-11-11 05:00:00+00:00  0.030086  0.033730  0.026255
2020-11-12 05:00:00+00:00 -0.002343 -0.008641 -0.005126
2020-11-13 05:00:00+00:00  0.000419  0.005958  0.004967
2020-11-16 05:00:00+00:00  0.008720  0.000719  0.003325
...                             ...       ...       ...
2023-11-02 04:00:00+00:00  0.020693  0.007810  0.006502
2023-11-03 04:00:00+00:00 -0.005181  0.003839  0.012862
2023-11-06 05:00:00+00:00  0.014605  0.008225  0.010573
2023-11-07 05:00:00+00:00  0.014451  0.021254  0.011219
2023-11-08 05:00:00+00:00  0.005885 -0.004415  0.007406

[754 rows x 3 columns]
AAPL mean: 0.0007580392192901268
AMZN mean: -0.0011414444220746664
MSFT mean: 0.0008290250824961204


In [39]:
# Calculate and display daily returns for 'SPY'
df_spy = alpaca.get_bars("SPY", timeframe, limit=None, start=start_date, end=end_date).df
daily_returns_spy = df_spy['close'].pct_change()
daily_returns_spy.dropna(inplace=True)
print("\nDaily Returns for SPY:")
print(daily_returns_spy)

# Calculate and display mean of daily return for 'SPY'
mean_spy = daily_returns_spy.mean()
print("\nSPY mean:")
print(mean_spy)


Daily Returns for SPY:
timestamp
2020-11-10 05:00:00+00:00   -0.001523
2020-11-11 05:00:00+00:00    0.007456
2020-11-12 05:00:00+00:00   -0.009475
2020-11-13 05:00:00+00:00    0.013471
2020-11-16 05:00:00+00:00    0.012510
                               ...   
2023-11-02 04:00:00+00:00    0.019164
2023-11-03 04:00:00+00:00    0.009123
2023-11-06 05:00:00+00:00    0.002300
2023-11-07 05:00:00+00:00    0.002846
2023-11-08 05:00:00+00:00    0.000732
Name: close, Length: 754, dtype: float64

SPY mean:
0.00033975975529095334


In [40]:
# Calculate and display standard deviation for each ticker
std_devs = {}
for stock in stock_tickers:
    if stock in daily_returns_tickers.columns:
        std_devs[stock] = daily_returns_tickers.std()[stock]
        print(f'{stock} standard deviation: {std_devs[stock]}')

AAPL standard deviation: 0.0177839401327225
AMZN standard deviation: 0.04183369731744983
MSFT standard deviation: 0.017581619238321928


In [41]:
# Calculate and display standard deviation for 'SPY'
std_dev_spy = daily_returns_spy.std()
print("\nStandard Deviation for SPY:")
print(std_dev_spy)


Standard Deviation for SPY:
0.011131185108019043


In [42]:
# Get the last day's closing prices for each ticker
last_day_closing_prices = df_combined_tickers.iloc[-1]

# Display the last day's closing prices
print("\nLast Day's Closing Prices:")
print(last_day_closing_prices)



Last Day's Closing Prices:
AAPL    182.89
AMZN    142.08
MSFT    363.20
Name: 2023-11-08 05:00:00+00:00, dtype: float64


In [45]:
# Get the last day's closing price for 'SPY'
spy_last_day_closing_price = df_spy['close'].iloc[-1]
print("\nLast Day's Closing Price for SPY:")
print(spy_last_day_closing_price)


Last Day's Closing Price for SPY:
437.25
